In [2]:
import yfinance as yf
import pandas as pd
from flask import Flask, request, jsonify
from flask_cors import CORS
from nltk.sentiment.vader import SentimentIntensityAnalyzer

app = Flask(__name__)
CORS(app)

# Downloading Tesla's stock price data
ticker = "TSLA"
start_date = "2019-03-16"
end_date = "2022-03-16"
tsla = yf.download(ticker, start=start_date, end=end_date)

# Reading Elon Musk's tweets data
elon_tweets = pd.read_csv("elonmusk_tweets.csv")
elon_tweets = elon_tweets[['date', 'tweet']]
elon_tweets['date'] = pd.to_datetime(elon_tweets['date'])

# Performing sentiment analysis on Elon Musk's tweets
analyzer = SentimentIntensityAnalyzer()
sentiments = []
for tweet in elon_tweets['tweet']:
    scores = analyzer.polarity_scores(tweet)
    sentiment = scores['compound']
    sentiments.append(sentiment)

elon_tweets['sentiment'] = sentiments
print('satya')
print(elon_tweets)
# Joining the sentiment scores with the stock price data
merged_df = tsla.merge(elon_tweets, how='left', left_index=True, right_on='date')
merged_df = merged_df.dropna()

# Creating a new column for the stock price movement
merged_df['price_movement'] = merged_df['Close'].diff().shift(-1)
merged_df['price_movement'] = merged_df['price_movement'].apply(lambda x: 1 if x > 0 else 0)

# Training a machine learning model to predict the stock price movement
X = merged_df[['sentiment', 'Close', 'Volume']]
y = merged_df['price_movement']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)


[*********************100%***********************]  1 of 1 completed
satya
            date                                              tweet  sentiment
0     2021-04-11  @vincent13031925 For now. Costs are decreasing...     0.0000
1     2021-04-11                           Love this beautiful shot     0.8442
2     2021-04-11  @agnostoxxx @CathieDWood @ARKInvest Trust the ...     0.5106
3     2021-04-11                 The art In Cyberpunk is incredible     0.0000
4     2021-04-11                                    @itsALLrisky 🤣🤣     0.0000
...          ...                                                ...        ...
12557 2021-04-14  @eugenelee3 @PPathole @SpaceX @Tesla Yeah, not...     0.7506
12558 2021-04-14  @PPathole @SpaceX @Tesla That was my night job...     0.0000
12559 2021-04-14  @PPathole @SpaceX @Tesla True. Ancient times …...     0.4215
12560 2021-04-14                  @Erdayastronaut @Tesla Absolutely     0.0000
12561 2021-04-13  @Erdayastronaut @Tesla Tesla is buildi

RandomForestClassifier(random_state=42)

In [9]:
new_tweet = 'tesla got new investers'
print(new_tweet)
scores = analyzer.polarity_scores(new_tweet)
sentiment = scores['compound']
last_close = merged_df['Close'].iloc[-1]
last_volume = merged_df['Volume'].iloc[-1]
X_new = pd.DataFrame([[sentiment, last_close, last_volume]], columns=['sentiment', 'Close', 'Volume'])
print(X_new)
prediction = clf.predict(X_new)[0]
print(last_close, prediction)
if prediction == 1:
    price_change = last_close * 0.01 # Replace 0.01 with the predicted percentage change
    response = {
        'prediction': 'The stock price is predicted to go up.',
        'price_change': f'The predicted price change is: {price_change:.2f}'
    }
else:
    price_change = last_close * -0.01 # Replace -0.01 with the predicted percentage change
    response = {
        'prediction': 'The stock price is predicted to go down.',
        'price_change': f'The predicted price change is: {price_change:.2f}'
    }

tesla got new investers
   sentiment       Close    Volume
0        0.0  246.593338  83938500
246.5933380126953 0


In [ ]:
scores

In [7]:
clf

RandomForestClassifier(random_state=42)